# 📰 Fake News Detection Using Ensemble Learning

---

| S. No |    ID No.     | Name             |
| ----: | :-----------: | :--------------- |
|    1. | 2022A7PS0003U | Yusra Hakim      |
|    2. | 2022A7PS0019U | Joseph Cijo      |
|    3. | 2022A7PS0031U | Ritvik Bhatnagar |

This Jupyter Notebook is for the project in Data Mining (CS F415) course. It contains the code used for preparing the ensemble model using the [dataset](#dataset-used) mentioned below.

### Dataset Details

**Dataset introduced in:**
V. Pawan Kumar, A. Prateek, A. Ivone and P. Radu, "WELFake: Word Embedding Over Linguistic Features for Fake News Detection," _IEEE Transactions on Computational Social Systems_, vol. 8, no. 4, pp. 881-893, 2021.
[Kaggle | WELFake Dataset](https://www.kaggle.com/datasets/saurabhshahane/fake-news-classification)

### Setting Up an Environment for the Jupyter notebook

Firstly install Miniconda from [here](https://docs.anaconda.com/miniconda/install/).

Then open a command prompt in this directory, and run the following. This will create and activate an environment called "PROJ".

```bash
    conda create -n proj python=3.12
    conda activate proj
```

After running this, your CMD prompt should have a "`(proj)`" prefixed at the start.

Run the following command to install packages, such as [PyTorch](https://pytorch.org/get-started/locally/). This will take some time.


In [ ]:
%conda install -n proj ipykernel ipywidgets --update-deps --force-reinstall
%conda install -n proj nltk
%conda install -n proj conda-forge::textblob
%pip install scikit-learn matplotlib pandas pyperclip contractions scipy numpy
%conda install -n proj conda-forge::transformers
%pip install torch --index-url https://download.pytorch.org/whl/cu124


Remember to select the PROJ environment at the bottom-right.

---

# 1. 📚 Outline

To train the model, the following steps must be followed:

<img src="archdiag.png" size=64/>

The model will accept two inputs:
1. The headline of the article
2. Content of the article (Optional)

And give an output if it is fake news or factual (real) news.

---

# 2. 📑 Dataset Tomfoolery

The dataset is a CSV file with the following columns:
- title: the title of the article
- text: the text of the article
- label: the label of the article (0 for fake, 1 for real)

The dataset used is the WELFake dataset [available here](https://www.kaggle.com/datasets/saurabhshahane/fake-news-classification)

This dataset is a collection of news articles, with each article labeled as either fake or real. The dataset contains 10,000 articles, with 5,000 fake and 5,000 real articles. The articles are in English and cover a wide range of topics, including politics, sports, entertainment, and technology.

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

import numpy as np

import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

from IPython.display import display, HTML
import ipywidgets as widgets

import random
import re
import contractions

from sklearn.feature_extraction.text import TfidfVectorizer
from transformers.models.distilbert import DistilBertTokenizer, DistilBertModel
import torch
import pickle


In [ ]:
df = pd.read_csv("dataset/WELFake_Dataset.csv", index_col=0)

df.head()


## ✂️ Splitting Dataset

The dataset is split as
-  80% → Training + Validation Set
-  20% → Testing Set

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


In [ ]:
print(f"Training set size: {train_df.shape}")
train_df.to_csv("dataset/train.csv", index=False)
train_df.head()


In [ ]:
print(f"\nTesting set size: {test_df.shape}")
test_df.to_csv("dataset/test.csv", index=False)
test_df.head()


## 👾 Data Preprocessing

The data is preprocessed using the following steps
1. [Text Cleaning](#text-cleaning)
   1. [Lowercasing & URL Removal](#lowercasing-url-removal)
   2. [Contractions Expansion](#contractions-expansion)
   3. [Tokenization](#tokenization)
   4. [Lemmatization](#lemmatization)
   5. [Stopword Removal](#stopword-removal)
2. [Data Augmentaiton](#data-augmentation)
   1. [Synonym Replacement](#synonym-replacement)
   2. [Random Insertion](#random-insertion)
   3. [Random Swap](#random-swap)
   4. [Random Deletion](#random-deletion)

---

### 🧹 Data Cleaning

The text in the datast has to be cleaned before it can be used for training the model. The following steps are used for cleaning the text.

[Kaggle | Getting started with Text Preprocessing](https://www.kaggle.com/code/sudalairajkumar/getting-started-with-text-preprocessing)


#### Merging Text Values

In [ ]:
def merge_title_and_text(df: pd.DataFrame) -> pd.DataFrame:
    """
    Merge the 'title' and 'text' fields into a single 'content' field.

    :param df: Input DataFrame
    :return: DataFrame with a new 'content' field
    """

    df["content"] = df["title"].fillna("") + " " + df["text"].fillna("")
    return df


#### Handling Missing Values

In [ ]:
def handle_missing_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Handle missing data in the DataFrame by removing rows with any empty, NaN values, or rows containing only whitespaces.

    :param df: DataFrame to process
    :return: DataFrame with rows containing any empty, NaN values, or whitespaces-only content removed
    """

    missing_values = df.isnull().sum()
    print("Missing values in each column:\n", missing_values)

    df = df.replace("", np.nan)

    df = df.dropna()

    return df


#### Lowercasing & URL Removal

The text is converted to lowercase and URLs are removed from the text.

In [ ]:
def lowercase_and_remove_urls(text: str) -> str:
    """
    Convert text to lowercase and remove URLs.

    :param text: Input text
    :return: Processed text with URLs removed
    """

    text = text.lower()

    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)
    return text


#### Contractions Expansion

Contractions are expanded to their full form. For example, "I'm" is expanded to "I am". Using the [`contractions`](https://github.com/kootenpv/contractions/tree/master) library for this purpose.

In [ ]:
def expand_contractions(text: str) -> str:
    """
    Expand contractions in the text using the `contractions` library.

    :param text: Input text
    :return: Text with contractions expanded
    """

    expanded_words = [contractions.fix(word) for word in text.split()]
    expanded_text = " ".join(expanded_words)  # type: ignore
    return expanded_text


#### Tokenization

The text is tokenized into words. This also removes punctuation and any special characters.

The [`RegexpTokenizer`](https://www.nltk.org/api/nltk.tokenize.RegexpTokenizer.html) tokenizer is used for tokenization by aplphanumeric words and to ignore all other characters.

In [ ]:
def tokenize_text(text: str) -> list:
    """
    Tokenize the input text using NLTK's RegexpTokenizer.

    :param text: Input text
    :return: List of tokens
    """

    tokenizer = RegexpTokenizer(r"\w+")
    tokens = tokenizer.tokenize(text)
    return tokens


#### Lemmatization

The words are lemmatized to their root form. This is done using the [`WordNetLemmatizer`](https://www.nltk.org/api/nltk.stem.html#nltk.stem.wordnet.WordNetLemmatizer) from the NLTK library.

In [ ]:
nltk.download("averaged_perceptron_tagger_eng")
nltk.download("wordnet")


In [ ]:
lemmatizer = WordNetLemmatizer()


def nltkToWordnet(nltk_tag: str) -> str:
    """
    Convert NLTK POS tags to WordNet POS tags.

    :param nltk_tag: NLTK POS tag
    :return: WordNet POS tag
    """

    if nltk_tag.startswith("J"):
        return wordnet.ADJ
    elif nltk_tag.startswith("V"):
        return wordnet.VERB
    elif nltk_tag.startswith("N"):
        return wordnet.NOUN
    elif nltk_tag.startswith("R"):
        return wordnet.ADV
    else:
        return None  # type: ignore


def lemmatize_text(tokens: list) -> list:
    """
    Lemmatize the tokens using NLTK's WordNetLemmatizer.

    :param tokens: List of tokens
    :return: List of lemmatized words
    """

    pos_tags = nltk.pos_tag(tokens)
    res_words = []
    for word, tag in pos_tags:
        tag = nltkToWordnet(tag)
        if tag is None:
            res_words.append(word)
        else:
            res_words.append(lemmatizer.lemmatize(word, tag))
    return res_words


#### Stopword Removal

Stopwords are removed from the text. Stopwords are common words that do not add much meaning to the text. The [`stopwords`](https://www.nltk.org/api/nltk.corpus.reader.html#nltk.corpus.reader.wordlist.WordListCorpusReader) from the NLTK library are used for this purpose.


In [ ]:
nltk.download("words")
nltk.download("stopwords")


In [ ]:
englishWords = set(nltk.corpus.words.words())
stop_words = set(stopwords.words("english"))


def remove_stopwords(tokens: list) -> str:
    """
    Remove stop words from the list of tokens.

    :param tokens: List of tokens
    :return: List of tokens without stop words
    """

    filtered_words = [w for w in tokens if (w in englishWords and w not in stop_words)]
    filtered_text = " ".join(filtered_words)
    return filtered_text


---

#### Performing Text Cleaning


Function to perform text cleaning on the dataset.

In [ ]:
def clean_text(text: str) -> str:
    """
    Clean the input text by applying various preprocessing steps.

    :param text: Input text
    :return: Cleaned text
    """
    text = lowercase_and_remove_urls(text)
    text = expand_contractions(text)
    tokens = tokenize_text(text)
    tokens = lemmatize_text(tokens)
    text = remove_stopwords(tokens)
    return text


In [ ]:
progress = widgets.IntProgress(
    value=0, min=0, max=len(train_df), description="Progress:"
)


In [ ]:
display(progress)

cleaned_data = []
train_df = merge_title_and_text(train_df)
train_df = handle_missing_data(train_df)

for index, row in train_df.iterrows():
    content = row["content"] if pd.notnull(row["content"]) else ""

    cleaned_content = clean_text(content)

    cleaned_data.append({"content": cleaned_content, "label": row["label"]})

    progress.value += 1

cleaned_train_df = pd.DataFrame(cleaned_data)
cleaned_train_df = handle_missing_data(cleaned_train_df)

cleaned_train_df.head()


---

### 🥀 Data Augmntation

Data augmentation is a technique used to increase the size of the dataset by adding slightly modified copies of the data. This is done to improve the performance of the model by providing more data for training.

Jason Wei, Kai Zou, "EDA: Easy Data Augmentation Techniques for Boosting Performance on Text Classification Tasks"
[Available here](https://arxiv.org/abs/1901.11196)

#### Synonym Replacement

In this technique, we replace n words in the sentence with synonyms from WordNet. We choose a random n words from the sentence that are not stop words. We then replace each of these words with a random synonym that is also present in the sentence.

In [ ]:
def get_synonyms(word: str) -> list:
    """
    Get synonyms for a given word using WordNet.

    :param word: Input word
    :return: List of synonyms
    """

    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():  # type: ignore
            synonyms.add(lemma.name())
    if word in synonyms:
        synonyms.remove(word)
    return list(synonyms)


def synonym_replacement(text: str, n: int) -> str:
    """
    Randomly replace words in the text with their synonyms.

    :param text: Input text
    :param n: Number of words to replace
    :return: Text with words replaced by synonyms
    """

    words = text.split()
    new_words = words.copy()
    random_word_list = list(set([word for word in words if wordnet.synsets(word)]))
    random.shuffle(random_word_list)
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(synonyms)
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n:
            break

    sentence = " ".join(new_words)

    return sentence


#### Random Insertion

In random insertion, we randomly insert synonyms of a word into the sentence n times. This is done by first finding a synonym of a word and then inserting it into the sentence at a random position.

In [ ]:
def add_word(word: str, sentence: str) -> str:
    """
    Add a word at a random position in the sentence.

    :param word: Word to add
    :param sentence: Original sentence
    :return: Sentence with the word added
    """

    words = sentence.split()
    random_idx = random.randint(0, len(words) - 1)
    words.insert(random_idx, word)
    return " ".join(words)


def random_insertion(text: str, n: int) -> str:
    """
    Randomly insert words into the text.

    :param text: Input text
    :param n: Number of words to insert
    :return: Text with words inserted
    """

    words = text.split()
    new_words = words.copy()
    for _ in range(n):
        random_word = random.choice(words)
        synonyms = get_synonyms(random_word)
        if synonyms:
            synonym = random.choice(synonyms)
            new_words = add_word(synonym, " ".join(new_words)).split()

    return " ".join(new_words)


#### Random Swap

In random swap, we randomly swap two words in the sentence n times. This is done by randomly choosing two words in the sentence and then swapping their positions.

In [ ]:
def random_swap(text: str, n: int) -> str:
    """
    Randomly swap words in the text.

    :param text: Input text
    :param n: Number of words to swap
    :return: Text with words swapped
    """

    words = text.split()
    new_words = words.copy()
    for _ in range(n):
        idx1, idx2 = random.sample(range(len(words)), 2)
        new_words[idx1], new_words[idx2] = new_words[idx2], new_words[idx1]

    return " ".join(new_words)


#### Random Deletion

In random deletion, we randomly delete each word in the sentence with a probability p.

In [ ]:
def random_deletion(text: str, p: float) -> str:
    """
    Randomly delete words from the text with a given probability.

    :param text: Input text
    :param p: Probability of deletion
    :return: Text with words deleted
    """

    words = text.split()
    if len(words) == 1:
        return text

    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:
            new_words.append(word)

    if len(new_words) == 0:
        return random.choice(words)

    return " ".join(new_words)


---

#### Performing Data Augmentation

In [ ]:
synonymReplaced_data = []
randomInserted_data = []
randomSwapped_data = []
randomDeleted_data = []


In [ ]:
progress = widgets.IntProgress(
    value=0, min=0, max=len(cleaned_train_df), description="Progress:"
)


In [ ]:
display(progress)

for index, row in cleaned_train_df.iterrows():
    content = row["content"] if pd.notnull(row["content"]) else row["content"]

    try:
        if pd.notnull(content):
            try:
                synonymReplaced_content = synonym_replacement(
                    content, (len(content.split()) // 3)
                )
            except Exception as e:
                print(f"Synonym replacement error for content: {e}")
                synonymReplaced_content = content

            try:
                randomInserted_content = random_insertion(
                    content, (len(content.split()) // 3)
                )
            except Exception as e:
                print(f"Random insertion error for content: {e}")
                randomInserted_content = content

            try:
                randomSwapped_content = random_swap(
                    content, (len(content.split()) // 3)
                )
            except Exception as e:
                print(f"Random swap error for content: {e}")
                randomSwapped_content = content

            try:
                randomDeleted_content = random_deletion(content, 0.1)
            except Exception as e:
                print(f"Random deletion error for content: {e}")
                randomDeleted_content = content
        else:
            synonymReplaced_content = content
            randomInserted_content = content
            randomSwapped_content = content
            randomDeleted_content = content
    except Exception as e:
        print(f"Augmentation error for content: {e}")
        synonymReplaced_content = content
        randomInserted_content = content
        randomSwapped_content = content
        randomDeleted_content = content

    synonymReplaced_data.append(
        {
            "content": synonymReplaced_content,
            "label": row["label"],
        }
    )

    randomInserted_data.append(
        {
            "content": randomInserted_content,
            "label": row["label"],
        }
    )

    randomSwapped_data.append(
        {
            "content": randomSwapped_content,
            "label": row["label"],
        }
    )

    randomDeleted_data.append(
        {
            "content": randomDeleted_content,
            "label": row["label"],
        }
    )

    progress.value += 1


In [ ]:
synonymReplaced_df = pd.DataFrame(synonymReplaced_data)
randomInserted_df = pd.DataFrame(randomInserted_data)
randomSwapped_df = pd.DataFrame(randomSwapped_data)
randomDeleted_df = pd.DataFrame(randomDeleted_data)

augmented_df = pd.concat(
    [synonymReplaced_df, randomInserted_df, randomSwapped_df, randomDeleted_df]
)

final_train_df = (
    pd.concat([cleaned_train_df, augmented_df]).sample(frac=1).reset_index(drop=True)
)


In [ ]:
print(f"Final training set size: {final_train_df.shape}")
final_train_df.to_csv("dataset/augmented_train.csv")
final_train_df.head()


---

### 🎭 Sentiment Analysis

TextBlob is a Python library for processing textual data. It provides a simple API for diving into common natural language processing (NLP) tasks such as part-of-speech tagging, noun phrase extraction, sentiment analysis, classification, translation, and more.

In [ ]:
from textblob import TextBlob


def analyze_sentiment(text: str) -> TextBlob.sentiment:  # type: ignore
    """
    Analyze the sentiment of the given text using TextBlob.

    :param text: Input text
    :return: Sentiment analysis result
    """

    blob = TextBlob(text)
    return blob.sentiment


In [ ]:
def add_sentiment(df: pd.DataFrame) -> pd.DataFrame:
    """
    Add sentiment analysis results (polarity and subjectivity) to the DataFrame.

    :param df: DataFrame to process
    :return: DataFrame with sentiment analysis results
    """

    progress = widgets.IntProgress(value=0, min=0, max=len(df), description="Progress:")
    display(progress)

    polarity = []
    subjectivity = []

    for _index, row in df.iterrows():
        sentiment = analyze_sentiment(row["content"])
        polarity.append(sentiment.polarity)
        subjectivity.append(sentiment.subjectivity)
        progress.value += 1

    df["polarity"] = polarity
    df["subjectivity"] = subjectivity
    df = df[["content", "polarity", "subjectivity", "label"]]

    return df


In [ ]:
final_train_df = add_sentiment(final_train_df)
final_train_df.to_csv("dataset/augmented_train_senti.csv", index=False)
final_train_df.head()


---

## 🔢 Feature Extraction

Extracting features for model training
- TF-IDF (for SVM)
- Word2Vec + GloVe (for RF and Gradient Boosting)
- SmallBERT (for LR)

In [2]:
final_train_df = pd.read_csv("dataset/augmented_train_senti.csv")


In [ ]:
# TF-IDF Feature Extraction
def extract_tfidf_features(df, max_features=5000):
    progress = widgets.IntProgress(value=0, min=0, max=1, description="TF-IDF:")
    display(progress)

    # Handle missing values in the 'content' column
    df["content"] = df["content"].fillna("")
    vectorizer = TfidfVectorizer(max_features=max_features)
    # tfidf_features = np.array(vectorizer.fit_transform(df["content"]))
    tfidf_features = vectorizer.fit_transform(df["content"])

    progress.value = 1
    return tfidf_features


In [ ]:
# GloVe Feature Extraction
def extract_glove_features(df, glove_path="glove.6B.100d.txt", embedding_dim=100):
    progress = widgets.IntProgress(value=0, min=0, max=len(df), description="GloVe:")
    display(progress)

    # Load GloVe embeddings
    glove_embeddings = {}
    with open(glove_path, "r", encoding="utf-8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype="float32")
            glove_embeddings[word] = vector

    # Compute sentence embeddings
    sentences = [content.split() for content in df["content"]]
    glove_features = np.array(
        [
            np.mean(
                [
                    glove_embeddings[word]
                    for word in sentence
                    if word in glove_embeddings
                ]
                or [np.zeros(embedding_dim)],
                axis=0,
            )
            for sentence in sentences
        ]
    )

    for _ in range(len(df)):
        progress.value += 1

    return glove_features


In [3]:
# SmallBERT Feature Extraction with Chunking
def extract_bert_features(df, model_name="distilbert-base-uncased", chunk_size=100):
    tokenizer = DistilBertTokenizer.from_pretrained(model_name)
    model = DistilBertModel.from_pretrained(model_name)
    bert_features = []

    progress = widgets.IntProgress(value=0, min=0, max=len(df), description="BERT:")
    display(progress)

    # Process the DataFrame in chunks
    for i in range(0, len(df), chunk_size):
        chunk = df.iloc[i : i + chunk_size]
        chunk_features = []

        for content in chunk["content"]:
            inputs = tokenizer(
                content,
                return_tensors="pt",
                truncation=True,
                padding=True,
                max_length=512,
            )
            with torch.no_grad():
                outputs = model(**inputs)
            cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
            chunk_features.append(cls_embedding)
            progress.value += 1

        bert_features.extend(chunk_features)

    return np.array(bert_features)


In [ ]:
tfidf_features = extract_tfidf_features(final_train_df)
with open("dataset/tfidf_vectorizer.pkl", "wb") as file:
    pickle.dump(tfidf_features, file)
del tfidf_features


In [ ]:
glove_features = extract_glove_features(final_train_df, glove_path="glove.6B.100d.txt")
with open("dataset/glove_vectorizer.pkl", "wb") as file:
    pickle.dump(glove_features, file)
del glove_features


In [4]:
bert_features = extract_bert_features(final_train_df)
with open("dataset/distilbert_vectorizer.pkl", "wb") as file:
    pickle.dump(bert_features, file)
del bert_features


IntProgress(value=0, description='BERT:', max=285640)

: 

In [ ]:
with open("dataset/tfidf_vectorizer.pkl", "rb") as file:
    loaded_vectorizer_pickle = pickle.load(file)
with open("dataset/GloVe.pkl", "rb") as file:
    loaded_glove_pickle = pickle.load(file)


---

## 📱 Display Sample Data

In [ ]:
def displayRandomSample(df: pd.DataFrame, dataType: int) -> None:
    """
    Display a random sample from the DataFrame.

    :param df: DataFrame to sample from
    :param dataType: Type of DataFrame to sample from (0 for original, 1 for augmented)
    """

    if dataType == 0:
        sample = df.sample(n=1)
        print(f"idx: {sample.index[0]}")
        text = sample.iloc[0]["text"]
        if isinstance(text, str):
            text = f"{text[:200]}..."
        else:
            text = "N/A"
        display(
            HTML(
                f"<b>Original:</b><br>"
                f"<b>Title:</b> {sample.iloc[0]['title']}<br>"
                f"<b>Text:</b> {text}<br>"
                f"<b>Label:</b> {sample.iloc[0]['label']}<br>"
            )
        )
    if dataType == 1:
        sample = df.sample(n=1)
        print(f"idx: {sample.index[0]}")
        text = sample.iloc[0]["content"]
        if isinstance(text, str):
            text = f"{text[:200]}..."
        else:
            text = "N/A"
        display(
            HTML(
                f"<b>Augmented:</b><br>"
                f"<b>Content:</b> {text}<br>"
                f"<b>Polarity:</b> {sample.iloc[0]['polarity']}<br>"
                f"<b>Subjectivity:</b> {sample.iloc[0]['subjectivity']}<br>"
                f"<b>Label:</b> {sample.iloc[0]['label']}"
            )
        )


In [ ]:
displayRandomSample(final_train_df, 1)
print(final_train_df.shape)


---

# 3. 🧠 Model Building




Preparing 5 models for ensemble learning model:
1. Random Forest (RF)
2. Support Vector Machine (SVM)
3. Gradient Boosting (XGBoost/LightGBM)
4. Logistic Regression (LR)
5. Small BERT (DistilBERT)